In [1]:
#import dependancies
import pandas as pd
import gmaps
import requests
# Import the API key.
from goog_config import goog_config

!jupyter nbextension enable --py --sys-prefix widgetsnbextension
!pip install gmaps
!jupyter nbextension enable --py --sys-prefix gmaps

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Hermanus,ZA,2022-11-14 00:16:48,-34.4187,19.2345,52.81,86,10,7.74
1,1,Kaeo,NZ,2022-11-14 00:16:48,-35.1000,173.7833,71.33,66,2,5.21
2,2,Jamestown,US,2022-11-14 00:12:23,42.0970,-79.2353,35.19,76,100,11.50
3,3,Dryden,CA,2022-11-14 00:16:49,49.7833,-92.7503,19.69,86,100,3.44
4,4,Georgetown,MY,2022-11-14 00:15:19,5.4112,100.3354,80.24,92,20,0.00


In [3]:
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [4]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=goog_config)

# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [7]:
# 1. Assign the locations to an array of latitude and longitude pairs.
locations = city_data_df[["Lat", "Lng"]]
# 2. Assign the weights variable to some values.
max_temp = city_data_df["Max Temp"]
# 3. Assign the figure variable to the gmaps.figure() attribute.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# 4. Assign the heatmap_layer variable to the heatmap_layer attribute and add in the locations.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4)

# 5. Add the heatmap layer.
fig.add_layer(heat_layer)
# 6. Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
# Heatmap of wind speed
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [10]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [11]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
4,4,Georgetown,MY,2022-11-14 00:15:19,5.4112,100.3354,80.24,92,20,0.00
6,6,Sao Joao Da Barra,BR,2022-11-14 00:16:50,-21.6403,-41.0511,75.06,89,84,16.89
7,7,Ca Mau,VN,2022-11-14 00:16:50,9.1769,105.1500,77.43,93,100,7.63
8,8,Isangel,VU,2022-11-14 00:16:51,-19.5500,169.2667,84.31,84,40,3.44
9,9,Cabo San Lucas,MX,2022-11-14 00:16:51,22.8909,-109.9124,79.90,64,42,1.01
13,13,Dingle,PH,2022-11-14 00:15:14,10.9995,122.6711,81.09,85,83,10.94
18,18,Atuona,PF,2022-11-14 00:16:56,-9.8000,-139.0333,77.83,75,1,16.67
24,24,Saint-Georges,GD,2022-11-14 00:12:30,12.0564,-61.7485,79.27,83,19,12.86
26,26,Kavieng,PG,2022-11-14 00:16:59,-2.5744,150.7967,85.68,67,99,14.65
29,29,Banda Aceh,ID,2022-11-14 00:17:00,5.5577,95.3222,76.06,89,100,3.58


In [12]:
preferred_cities_df.count()

City_ID       116
City          116
Country       116
Date          116
Lat           116
Lng           116
Max Temp      116
Humidity      116
Cloudiness    116
Wind Speed    116
dtype: int64

In [13]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
4,Georgetown,MY,80.24,5.4112,100.3354,
6,Sao Joao Da Barra,BR,75.06,-21.6403,-41.0511,
7,Ca Mau,VN,77.43,9.1769,105.1500,
8,Isangel,VU,84.31,-19.5500,169.2667,
9,Cabo San Lucas,MX,79.90,22.8909,-109.9124,
13,Dingle,PH,81.09,10.9995,122.6711,
18,Atuona,PF,77.83,-9.8000,-139.0333,
24,Saint-Georges,GD,79.27,12.0564,-61.7485,
26,Kavieng,PG,85.68,-2.5744,150.7967,
29,Banda Aceh,ID,76.06,5.5577,95.3222,


In [15]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": goog_config
}

In [24]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [26]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
4,Georgetown,MY,80.24,5.4112,100.3354,Cititel Penang
6,Sao Joao Da Barra,BR,75.06,-21.6403,-41.0511,Pousada Porto De Canoas
7,Ca Mau,VN,77.43,9.1769,105.1500,Mường Thanh Luxury Cà Mau Hotel
8,Isangel,VU,84.31,-19.5500,169.2667,Tanna Lodge
9,Cabo San Lucas,MX,79.90,22.8909,-109.9124,Hotel Tesoro Los Cabos
13,Dingle,PH,81.09,10.9995,122.6711,SEGAYA PROPERTY
18,Atuona,PF,77.83,-9.8000,-139.0333,Villa Enata
24,Saint-Georges,GD,79.27,12.0564,-61.7485,Radisson Grenada Beach Resort
26,Kavieng,PG,85.68,-2.5744,150.7967,Nusa Island Retreat
29,Banda Aceh,ID,76.06,5.5577,95.3222,OYO 884 Rumoh PMI Hotel


In [30]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [31]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [32]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))